In [3]:
# Get an A100 high memory instance Premium +
# Get repo, change to bin directory, 
!git lfs install
!git clone https://github.com/retospect/proteoxystis.git
%cd ~
%cd proteoxystis/bin
!git checkout 16bitfloat
# install dependencies
!pip install -r requirements.txt
# Prep data
!python3 ./pickelize.py

Error: Failed to call git rev-parse --git-dir: exit status 128 
Git LFS initialized.
Cloning into 'proteoxystis'...
remote: Enumerating objects: 336, done.
remote: Counting objects: 100% (199/199), done.
remote: Compressing objects: 100% (122/122), done.
remote: Total 336 (delta 152), reused 115 (delta 72), pack-reused 137
Receiving objects: 100% (336/336), 44.56 MiB | 18.21 MiB/s, done.
Resolving deltas: 100% (233/233), done.
Filtering content: 100% (3/3), 54.17 MiB | 22.68 MiB/s, done.
/root
/root/proteoxystis/bin
Branch '16bitfloat' set up to track remote branch '16bitfloat' from 'origin'.
Switched to a new branch '16bitfloat'
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.6/205.6 kB 21.0 MB/s eta 0:00:00
Reading toml file
Removing stupid data
 99% 70287/70896 [00:00<00:00, 90336.47it/s]
after removing stupid data, this many remain  70287
Input array size (pdb x one-hot sequence):  (7

From class example:


In [4]:
# Load data:
import pickle

print("Loading data...", end="", flush=True)
# gzip the pickle is possible but way slow
with open("training_data.pickle", "rb") as f:
    (
        metadata,
        seqs,
        output,
        seqs_test,
        output_test,
        relevant_train,
        relevant_test,
    ) = pickle.load(f)
print("done")



Loading data...done


In [5]:
import torch
import torch.nn as nn

# Set the random seed for reproducible results
torch.manual_seed(1)

class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        # This just calls the base class constructor
        super().__init__()
        # Neural network layers assigned as attributes of a Module subclass
        # have their parameters registered for training automatically.
        self.rnn = torch.nn.RNN(input_size, hidden_size, nonlinearity='relu', batch_first=True)
        self.linear = torch.nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # The RNN also returns its hidden state but we don't use it.
        # While the RNN can also take a hidden state as input, the RNN
        # gets passed a hidden state initialized with zeros by default.
        h = self.rnn(x)[0]
        x = self.linear(h)
        return x

class SimpleLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.lstm = torch.nn.LSTM(input_size, hidden_size, num_layers=7,batch_first=True,dropout=0.3)
        self.linear = torch.nn.Linear(hidden_size, hidden_size)
        self.linear1 = torch.nn.Linear(hidden_size, hidden_size)
        self.linear2 = torch.nn.Linear(hidden_size, hidden_size)
        self.linear3 = torch.nn.Linear(hidden_size, hidden_size)
        self.drop = torch.nn.Dropout(0.25)
        self.drop0 = torch.nn.Dropout(0.25)
        self.softmax1 = torch.nn.Softmax()
        self.softmax2 = torch.nn.Softmax()
        self.linear4 = torch.nn.Linear(hidden_size, output_size)
        print("Hidden size", hidden_size)
        print("output size", output_size)
        print("input size", input_size)

    def forward(self, x):
        h = self.lstm(x)[0]
        x = self.linear(h)
        x = self.linear1(x)
        x = self.linear2(x)
        x = self.linear3(x)
        x = self.linear4(x)
        return x
    
    def get_states_across_time(self, x):
        h_c = None
        h_list, c_list = list(), list()
        with torch.no_grad():
            for t in range(x.size(1)):
                h_c = self.lstm(x[:, [t], :], h_c)[1]
                h_list.append(h_c[0])
                c_list.append(h_c[1])
            h = torch.cat(h_list)
            c = torch.cat(c_list)
        return h, c

In [14]:
def train(model, train_data_gen, criterion, optimizer, device):
    # Set the model to training mode. This will turn on layers that would
    # otherwise behave differently during evaluation, such as dropout.
    model.train()

    # Store the number of sequences that were classified correctly
    num_correct = 0

    # Iterate over every batch of sequences. Note that the length of a data generator
    # is defined as the number of batches required to produce a total of roughly 1000
    # sequences given a batch size.
    for batch_idx in range(len(train_data_gen)):
        if batch_idx%25 ==0:
          print(",", end="", flush=True)
        # Request a batch of sequences and class labels, convert them into tensors
        # of the correct type, and then send them to the appropriate device.
        data, target, mask = train_data_gen[batch_idx]
        data, target, mask = torch.from_numpy(data).float().to(device), torch.from_numpy(target).float().to(device), torch.from_numpy(mask).float().to(device)

        # Perform the forward pass of the model
        output = model(data)  # Step ①

        # Pick only the output corresponding to last sequence element (input is pre padded)
        output = output[:, -1, :]
        #output = mask*output#
        # Compute the value of the loss for this batch. For loss functions like CrossEntropyLoss,
        # the second argument is actually expected to be a tensor of class indices rather than
        # one-hot encoded class labels. One approach is to take advantage of the one-hot encoding
        # of the target and call argmax along its second dimension to create a tensor of shape
        # (batch_size) containing the index of the class label that was hot for each sequence.
        target = target.argmax(dim=1)
        #print(output.shape, target.shape)
        loss = criterion(output, target)  # Step ②

        # Clear the gradient buffers of the optimized parameters.
        # Otherwise, gradients from the previous batch would be accumulated.
        optimizer.zero_grad()  # Step ③

        loss.backward()  # Step ④

        optimizer.step()  # Step ⑤

        y_pred = output.argmax(dim=1)
        num_correct += (y_pred == target).sum().item()
    print(":", end="", flush=True)
    return num_correct, loss.item()

In [7]:
def test(model, test_data_gen, criterion, device):
    # Set the model to evaluation mode. This will turn off layers that would
    # otherwise behave differently during training, such as dropout.
    model.eval()

    # Store the number of sequences that were classified correctly
    num_correct = 0

    # A context manager is used to disable gradient calculations during inference
    # to reduce memory usage, as we typically don't need the gradients at this point.
    with torch.no_grad():
        for batch_idx in range(len(test_data_gen)):
            data, target, mask = test_data_gen[batch_idx]
            data, target, mask = torch.from_numpy(data).float().to(device), torch.from_numpy(target).float().to(device), torch.from_numpy(mask).float().to(device)

            output = model(data)
            # Pick only the output corresponding to last sequence element (input is pre padded)
            output = output[:, -1, :]
            #output =mask*output
            target = target.argmax(dim=1)
            loss = criterion(output, target)

            y_pred = output.argmax(dim=1)
            num_correct += (y_pred == target).sum().item()

    return num_correct, loss.item()

In [8]:
import matplotlib.pyplot as plt
#from res.plot_lib import set_default, plot_state, print_colourbar
#set_default()

In [9]:
def train_and_test(model, train_data_gen, test_data_gen, criterion, optimizer, max_epochs, verbose=True):
    # Automatically determine the device that PyTorch should use for computation
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    # Move model to the device which will be used for train and test
    model.to(device)

    # Track the value of the loss function and model accuracy across epochs
    history_train = {'loss': [], 'acc': []}
    history_test = {'loss': [], 'acc': []}

    for epoch in range(max_epochs):
        # Run the training loop and calculate the accuracy.
        # Remember that the length of a data generator is the number of batches,
        # so we multiply it by the batch size to recover the total number of sequences.
        num_correct, loss = train(model, train_data_gen, criterion, optimizer, device)
        accuracy = float(num_correct) / (len(train_data_gen) )* len(train_data_gen) * 100 #fixme
        history_train['loss'].append(loss)
        history_train['acc'].append(accuracy)

        # Do the same for the testing loop
        num_correct, loss = test(model, test_data_gen, criterion, device)
        accuracy = float(num_correct) / (len(test_data_gen) ) * len(test_data_gen) * 100 #FIXME
        history_test['loss'].append(loss)
        history_test['acc'].append(accuracy)

        print(f'\n[Epoch {epoch + 1}/{max_epochs}]'
                  f" loss: {history_train['loss'][-1]:.4f}, acc: {history_train['acc'][-1]:2.2f}%"
                  f" - test_loss: {history_test['loss'][-1]:.4f}, test_acc: {history_test['acc'][-1]:2.2f}%")

    # Generate diagnostic plots for the loss and accuracy
    fig, axes = plt.subplots(ncols=2, figsize=(9, 4.5))
    for ax, metric in zip(axes, ['loss', 'acc']):
        ax.plot(history_train[metric])
        ax.plot(history_test[metric])
        ax.set_xlabel('epoch', fontsize=12)
        ax.set_ylabel(metric, fontsize=12)
        ax.legend(['Train', 'Test'], loc='best')
    plt.show()

    return model

In [11]:
import numpy as np
# Setup the training and test data generators
        # seqs,
        # output,
        # seqs_test,
        # output_test,
        # relevant_test,
        # relevant_train,
from torch.utils.data import TensorDataset, DataLoader
import torch
setlen = 256
print("len",len(seqs))
numpieces = len(seqs)//setlen
longbit = numpieces*setlen
print("pcs",numpieces)
print(len(seqs))
print(longbit)
seqs2,_ = np.split(seqs, [longbit])
output2,_ = np.split(output, [longbit])
relevant_train2,_ = np.split(relevant_train, [longbit])
seqs2 = np.split(seqs2,setlen)
output2 = np.split(output2, setlen)
relevant_train2 = np.split(relevant_train2, setlen)

print(len(output2))
trainy = []
for i in range(setlen):
  trainy.append((seqs2[i], output2[i], relevant_train2[i]))

## Test

setlen = 256
print(len(seqs_test))
numpieces = len(seqs_test)//setlen
longbit = numpieces*setlen
print(numpieces)
print(len(seqs_test))
print(longbit)
seqs2,_ = np.split(seqs_test, [longbit])
output2,_ = np.split(output_test, [longbit])
relevant_train2,_ = np.split(relevant_train, [longbit])
seqs2 = np.split(seqs2,setlen)
output2 = np.split(output2, setlen)
relevant_test2,_ = np.split(relevant_train2, [longbit])
print(len(output2))
testy = []
for i in range(setlen):
  testy.append((seqs2[i], output2[i], relevant_test2[i]))


len 52716
pcs 205
52716
52480
256
17571
68
17571
17408
256


In [ ]:
input_size = 4128
# Setup the RNN and training settings
input_size  = 21 
hidden_size = 16
output_size = 698
model       = SimpleLSTM(input_size, hidden_size, output_size)
criterion   = torch.nn.CrossEntropyLoss()#torch.nn.MSELoss()#
optimizer   = torch.optim.RMSprop(model.parameters(), lr=0.001)
max_epochs  = 10

# Train the model
model = train_and_test(model, trainy, testy, criterion, optimizer, max_epochs, verbose=False)

Hidden size 16
output size 698
input size 21
,,,